In [ ]:
!pip install transformers
!pip install mlflow
!pip install evaluate
!pip install wandb
!pip install --upgrade accelerate

In [2]:
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from imblearn.datasets import make_imbalance
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer, BertModel
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset, load_from_disk, DatasetDict
import evaluate
import os
import io

In [3]:
import wandb

In [4]:
# Login to W&B
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
wandb.init(
    # set the wandb project where this run will be logged
    project="01_06")

wandb: Currently logged in as: k-kjoerup (ccs-project). Use `wandb login --relogin` to force relogin


In [10]:
# Load model and tokenizer

tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# import data
data = pd.read_csv('text_emoji.csv', index_col = 0)

In [ ]:
# Delete party column
#data = data.drop(['party'], axis = 1)

In [13]:
data = data.astype({'label': 'int32'})
#type(data['label'].iloc[0])

### Fine-tuning model

In [14]:
# split data

dataset = Dataset.from_pandas(data, preserve_index=False)

train_test = dataset.train_test_split(shuffle = True, seed = 200, test_size=0.3)
test_val = train_test['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)

train_val_test_dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_val['test'],
    'dev': test_val['train']})

In [15]:
#Tokenize data

def tokenize_function(examples):
    return tokenizer(examples["comment_text"], padding="max_length", truncation=True,max_length=128)
  
tokenized_datasets = train_val_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/814 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

In [16]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
#Load the training arguments and hyperparameters
training_args = TrainingArguments(output_dir="/content/roberta_model",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=20,
                                  save_strategy = "epoch",
                                  load_best_model_at_end=True,
                                  learning_rate=1e-6,
                                  report_to="wandb",
                                  # args to prevent disk overload on Google Colab
                                  save_steps = -1,
                                  save_on_each_node = False,
                                  overwrite_output_dir = True
                                  )

In [18]:
#Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.557300,0.504148,0.793103
2,0.513900,0.485776,0.804598
3,0.463000,0.467872,0.810345
4,0.453800,0.458676,0.810345
5,0.439400,0.451624,0.816092
6,0.412200,0.443227,0.816092
7,0.407400,0.439675,0.821839
8,0.385600,0.440318,0.827586
9,0.399900,0.438961,0.827586
10,0.370000,0.438055,0.827586


eval/accuracy,▁▃▃▃▄▄▅▆▆▆▆▆▆▆▆█▇▇▇▇
eval/loss,█▆▄▃▂▂▁▁▁▁▁▂▂▂▂▃▃▃▃▃
eval/runtime,▁▁▃▄▅█▆▆▆▆▇▆▆▆▇▆▆█▆▆
eval/samples_per_second,██▆▅▃▁▂▃▃▃▂▃▃▂▂▃▂▁▃▃
eval/steps_per_second,██▆▅▃▁▂▃▃▃▂▃▃▂▂▃▂▁▃▃
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▇▅▅▅▄▄▃▄▃▂▂▂▂▂▂▂▁▁▁
train/total_flos,▁
train/train_loss,▁


# Make predictions

In [19]:
predictions = trainer.predict(tokenized_datasets['test'])

In [20]:
#test metrics
predictions[2]

{'test_loss': 0.4905324876308441,
 'test_accuracy': 0.7828571428571428,
 'test_runtime': 1.4058,
 'test_samples_per_second': 124.487,
 'test_steps_per_second': 7.825}

In [21]:
pred_labels = np.array([prediction.argmax() for prediction in predictions[0]])

In [22]:
predictions[1]

array([0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2,
       2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 2, 1, 0, 1, 2, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 1, 1, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       2, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 2, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 2])

In [23]:
pred_labels

array([0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2,
       2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 2, 1, 2, 1,
       0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 2])

In [24]:
test_df = pd.DataFrame(train_val_test_dataset['test'])

In [25]:

#this shows the distribution of annotated labels within each party
test_df.groupby('party')['label'].value_counts()

party          label
conservatives  0        57
               1        16
               2        13
labour         0        30
               1        19
               2        14
libdems        0        18
               2         5
               1         3
Name: label, dtype: int64

In [26]:
test_df['predicted_label'] = pred_labels

In [27]:
#this shows the distribution of predicted labels within each party
test_df.groupby('party')['predicted_label'].value_counts()

party          predicted_label
conservatives  0                  60
               2                  15
               1                  11
labour         0                  41
               2                  19
               1                   3
libdems        0                  17
               2                   5
               1                   4
Name: predicted_label, dtype: int64

In [28]:
#conservatives mean sentiment (just taking the mean of all the predicted labels)
test_df[test_df['party'] == 'conservatives']['predicted_label'].mean()

0.47674418604651164

In [29]:
#labour mean sentiment
test_df[test_df['party'] == 'labour']['predicted_label'].mean()

0.6507936507936508

In [30]:
#libdems mean sentiment
test_df[test_df['party'] == 'libdems']['predicted_label'].mean()

0.5384615384615384